In [1]:
import climetlab
import xarray as xr
import climpred

In [2]:
dates = xr.cftime_range(start="2020-01-02", freq="7D", end="2020-12-31")
var = "t2m"
date = dates[0].strftime("%Y%m%d")
date

'20200102'

In [3]:
import intake
import fsspec
from aiohttp import ClientSession, ClientTimeout

timeout = ClientTimeout(total=600)
fsspec.config.conf["https"] = dict(client_kwargs={"timeout": timeout})

import intake_xarray

cache_path = "my_caching_folder"
fsspec.config.conf["simplecache"] = {"cache_storage": cache_path, "same_names": True}

In [4]:
forecast_climetlab = (
    climetlab.load_dataset(
        "s2s-ai-challenge-test-input",
        origin="ecmwf",
        date=[20200102, 20200206],  # get to initializations/forecast_times
        parameter=var,
        format="netcdf",
    )
    .to_xarray()
    .compute()
)

By downloading data from this dataset, you agree to the terms and conditions defined at https://apps.ecmwf.int/datasets/data/s2s/licence/. If you do not agree with such terms, do not download the data. 


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
forecast_times = forecast_climetlab.set_coords("valid_time").valid_time.values.flatten()

# takes a while, continue with obs from above
obs_era = (
    climetlab.load_source(
        "cds",
        "reanalysis-era5-single-levels",
        product_type="reanalysis",
        time=["00:00"],
        grid=[1.5, 1.5],
        param="2t",
        date=forecast_times,
    )
    .to_xarray()
    .squeeze(drop=True)
)

yeechian's Corpernicus API key: 23530:24a39ef1-0eb5-4632-8475-4a3bca72088c

In [6]:
fct = (
    climpred.HindcastEnsemble(
        forecast_climetlab.drop_vars("valid_time").isel(lead_time=range(4))
    )
    .add_observations(obs_era)
    .compute()
)

metric_kwargs = dict(
    metric="rmse", comparison="e2o", alignment="same_inits", dim="init"
)

C:\Users\klow55\Anaconda3\lib\site-packages\climpred\checks.py:256: UserWarning: Did not find dimension "init", but renamed dimension forecast_time with CF-complying standard_name "forecast_reference_time" to init.
  warnings.warn(
C:\Users\klow55\Anaconda3\lib\site-packages\climpred\checks.py:256: UserWarning: Did not find dimension "member", but renamed dimension realization with CF-complying standard_name "realization" to member.
  warnings.warn(
C:\Users\klow55\Anaconda3\lib\site-packages\climpred\checks.py:256: UserWarning: Did not find dimension "lead", but renamed dimension lead_time with CF-complying standard_name "forecast_period" to lead.
  warnings.warn(


In [7]:
fct

climpred.HindcastEnsemble

<Initialized Ensemble>
Dimensions: (member: 51, init: 2, lead: 4, latitude: 121, longitude: 240)
Coordinates:
 * member (member) int64 0 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
 * init (init) object 2020-01-02 00:00:00 2020-02-06 00:00:00
 * lead (lead) float64 1.0 2.0 3.0 4.0
 * latitude (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
 * longitude (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
Data variables:
 t2m (member, init, lead, latitude, longitude) float32 244.4 ... 238.7
Attributes:
 GRIB_edition: 2
 GRIB_centre: ecmf
 GRIB_centreDescription: European Centre for Medium-Range Weather Forecasts
 GRIB_subCentre: 0
 Conventions: CF-1.7
 institution: European Centre for Medium-Range Weather Forecasts Initialized Ensemble Dimensions: member : 51 init : 2 lead : 4 latitude : 121 longitude : 240 Coordinates: (5) member (member) int64 0 1 2 3 4 5 6 ... 45 46 47 48 49 50 long_name : Member units : 1 standard_name : realization description : Realization is used to label a dimension that can be thought of as a statistical sample, e.g., labelling members of a model ensemble. array([ 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50], dtype=int64) init (init) object 2020-01-02 00:00:00 2020-02-06 0... standard_name : forecast_reference_time long_name : Initialization description : The forecast reference time in NWP is the "data time", the time of the analysis from which the forecast was made. It is not the time for which the forecast is valid; the standard name of time should be used for that time. array([cftime.DatetimeProlepticGregorian(2020, 1, 2, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 6, 0, 0, 0, 0, has_year_zero=True)],
 dtype=object) lead (lead) float64 1.0 2.0 3.0 4.0 units : days standard_name : forecast_period long_name : Lead description : Forecast period is the time interval between the forecast reference time and the validity time. A period is an interval of time, or the time-period of an oscillation. array([1., 2., 3., 4.]) latitude (latitude) float64 90.0 88.5 87.0 ... -88.5 -90.0 units : degrees_north standard_name : latitude long_name : latitude stored_direction : decreasing amip : latitude description : Latitude is positive northward; its units of degree_north (or equivalent) indicate this explicitly. In a latitude-longitude system defined with respect to a rotated North Pole, the standard name of grid_latitude should be used instead of latitude. Grid latitude is positive in the grid-northward direction, but its units should be plain degree. array([ 90. , 88.5, 87. , 85.5, 84. , 82.5, 81. , 79.5, 78. , 76.5,
 75. , 73.5, 72. , 70.5, 69. , 67.5, 66. , 64.5, 63. , 61.5,
 60. , 58.5, 57. , 55.5, 54. , 52.5, 51. , 49.5, 48. , 46.5,
 45. , 43.5, 42. , 40.5, 39. , 37.5, 36. , 34.5, 33. , 31.5,
 30. , 28.5, 27. , 25.5, 24. , 22.5, 21. , 19.5, 18. , 16.5,
 15. , 13.5, 12. , 10.5, 9. , 7.5, 6. , 4.5, 3. , 1.5,
 0. , -1.5, -3. , -4.5, -6. , -7.5, -9. , -10.5, -12. , -13.5,
 -15. , -16.5, -18. , -19.5, -21. , -22.5, -24. , -25.5, -27. , -28.5,
 -30. , -31.5, -33. , -34.5, -36. , -37.5, -39. , -40.5, -42. , -43.5,
 -45. , -46.5, -48. , -49.5, -51. , -52.5, -54. , -55.5, -57. , -58.5,
 -60. , -61.5, -63. , -64.5, -66. , -67.5, -69. , -70.5, -72. , -73.5,
 -75. , -76.5, -78. , -79.5, -81. , -82.5, -84. , -85.5, -87. , -88.5,
 -90. ]) longitude (longitude) float64 0.0 1.5 3.0 ... 355.5 357.0 358.5 units : degrees_east standard_name : longitude long_name : longitude amip : longitude description : Longitude is positive eastward; its units of degree_east (or equivalent) indicate this explicitly. In a latitude-longitude system defined with respect to a rotated North Pole, the standard name of grid_longitude should be used instead of longitude. Grid longitude is positive in the grid-eastward direction, but its units 

<Observations>
Dimensions: (time: 81, latitude: 121, longitude: 240)
Coordinates:
 * time (time) object 2020-01-02 00:00:00 ... 2020-03-22 00:00:00
 * latitude (latitude) float64 90.0 88.5 87.0 85.5 ... -87.0 -88.5 -90.0
 * longitude (longitude) float64 0.0 1.5 3.0 4.5 ... 354.0 355.5 357.0 358.5
 valid_time (time) datetime64[ns] 2020-01-02 2020-01-03 ... 2020-03-22
Data variables:
 t2m (time, latitude, longitude) float32 244.3 244.3 ... 225.3 225.3
Attributes:
 GRIB_edition: 1
 GRIB_centre: ecmf
 GRIB_centreDescription: European Centre for Medium-Range Weather Forecasts
 GRIB_subCentre: 0
 Conventions: CF-1.7
 institution: European Centre for Medium-Range Weather Forecasts
 history: 2022-12-24T15:59 GRIB to CDM+CF via cfgrib-0.9.1... Observations Dimensions: time : 81 latitude : 121 longitude : 240 Coordinates: (4) time (time) object 2020-01-02 00:00:00 ... 2020-03-... array([cftime.DatetimeProlepticGregorian(2020, 1, 2, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 3, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 4, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 5, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 6, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 7, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 8, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 9, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 10, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 11, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 12, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 13, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 14, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 15, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 16, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 17, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 18, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 19, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 20, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 21, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 22, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 23, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 24, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 25, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 26, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 27, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 28, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 29, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 30, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 1, 31, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 1, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 2, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 3, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 4, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 5, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 6, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 7, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 8, 0, 0, 0, 0, has_year_zero=True),
 cftime.DatetimeProlepticGregorian(2020, 2, 9, 0,

In [9]:
import cupy

In [11]:
cupy.__file__

'C:\\Users\\klow55\\Anaconda3\\lib\\site-packages\\cupy\\__init__.py'